## ITP4514 - Lab 9 - Answers

### 1a)

In [1]:
# Load data
import pandas as pd

df = pd.read_csv('tweet_data.csv')
df.head()

id  label                                              tweet
0   1      0   @user when a father is dysfunctional and is s...
1   2      0  @user @user thanks for #lyft credit i can't us...
2   3      0                                bihday your majesty
3   4      0  #model   i love u take with u all the time in ...
4   5      0             factsguide: society now    #motivation

In [2]:
# Lower Case
df['tweet'] = df['tweet'].apply(lambda x: " ".join(x.lower() \
                                for x in x.split()))
df['tweet'].head()

0    @user when a father is dysfunctional and is so...
1    @user @user thanks for #lyft credit i can't us...
2                                  bihday your majesty
3    #model i love u take with u all the time in ur...
4                  factsguide: society now #motivation
Name: tweet, dtype: object

In [3]:
# Removing Punctuation
df['tweet'] = df['tweet'].str.replace('[^\w\s]','')
df['tweet'].head()

0    user when a father is dysfunctional and is so ...
1    user user thanks for lyft credit i cant use ca...
2                                  bihday your majesty
3    model i love u take with u all the time in urð...
4                    factsguide society now motivation
Name: tweet, dtype: object

In [4]:
# Removing accented characters
import unicodedata

df['tweet'] = df['tweet'].apply(lambda x: unicodedata.normalize('NFKD', x).\
                                encode('ascii', 'ignore').decode('utf-8', 'ignore'))
df['tweet'].head()

0    user when a father is dysfunctional and is so ...
1    user user thanks for lyft credit i cant use ca...
2                                  bihday your majesty
3       model i love u take with u all the time in ur 
4                    factsguide society now motivation
Name: tweet, dtype: object

In [5]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\johnnyc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
stop = stopwords.words('english')
df['tweet'] = df['tweet'].apply(lambda x: \
                        " ".join(x for x in x.split() if x not in stop))
df['tweet'].head()

0    user father dysfunctional selfish drags kids d...
1    user user thanks lyft credit cant use cause do...
2                                       bihday majesty
3                          model love u take u time ur
4                        factsguide society motivation
Name: tweet, dtype: object

In [7]:
# Lemmatization
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\johnnyc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
lem = WordNetLemmatizer()
df['tweet'][:5].apply(lambda x: " "\
                    .join([lem.lemmatize(word, "v") \
                        for word in x.split()]))


df['tweet'].to_csv("new_dogs_with_bmi.csv")

### 1b)

In [9]:
# Term frequency (TF)
tf1 = (df['tweet'][1:]).apply(lambda x: \
                    pd.value_counts(x.split(" ")))\
                    .sum(axis = 0).reset_index()
tf1.columns = ['words', 'tf']
tf1.head()


words      tf
0    user  1029.0
1   cause     5.0
2   offer     3.0
3  thanks    15.0
4    dont    47.0

In [10]:
# Inverse Document Frequency (IDF)
import numpy as np

for i,word in enumerate(tf1['words']):
    tf1.loc[i, 'idf'] = np.log(df.shape[0]/(len(df[df['tweet']\
                        .str.contains(word)])))

tf1.head()

words      tf       idf
0    user  1029.0  1.098112
1   cause     5.0  5.298317
2   offer     3.0  6.214608
3  thanks    15.0  4.710531
4    dont    47.0  3.669077

### 1c)

In [11]:
# List Bigrams
bigrams = [b for l in df['tweet'] for b in zip(l.split(" ")[:-1], l.split(" ")[1:])]
print(bigrams)

[('user', 'father'), ('father', 'dysfunctional'), ('dysfunctional', 'selfish'), ('selfish', 'drags'), ('drags', 'kids'), ('kids', 'dysfunction'), ('dysfunction', 'run'), ('user', 'user'), ('user', 'thanks'), ('thanks', 'lyft'), ('lyft', 'credit'), ('credit', 'cant'), ('cant', 'use'), ('use', 'cause'), ('cause', 'dont'), ('dont', 'offer'), ('offer', 'wheelchair'), ('wheelchair', 'vans'), ('vans', 'pdx'), ('pdx', 'disapointed'), ('disapointed', 'getthanked'), ('bihday', 'majesty'), ('model', 'love'), ('love', 'u'), ('u', 'take'), ('take', 'u'), ('u', 'time'), ('time', 'ur'), ('factsguide', 'society'), ('society', 'motivation'), ('22', 'huge'), ('huge', 'fan'), ('fan', 'fare'), ('fare', 'big'), ('big', 'talking'), ('talking', 'leave'), ('leave', 'chaos'), ('chaos', 'pay'), ('pay', 'disputes'), ('disputes', 'get'), ('get', 'allshowandnogo'), ('user', 'camping'), ('camping', 'tomorrow'), ('tomorrow', 'user'), ('user', 'user'), ('user', 'user'), ('user', 'user'), ('user', 'user'), ('user', '

### 2a)

In [12]:
# Prepare Data
s = df['tweet'][:2000]
text = ""
for _, t in s.items():
    text += t + " "

text
words = text.split()
fd = nltk.FreqDist(words)
fd.most_common(3)


[('user', 1030), ('love', 174), ('day', 125)]

### 2b)

In [13]:
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
sia.polarity_scores(text)


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\johnnyc\AppData\Roaming\nltk_data...


{'neg': 0.101, 'neu': 0.608, 'pos': 0.291, 'compound': 1.0}

### 2c)

In [14]:
# Sentiment Analysis Using NLTK
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import names
 
def word_feats(word):
    return dict([(char, True) for char in word])
 
positive_vocab =  pd.read_csv('positiveWords.csv')
negative_vocab = pd.read_csv('negativeWords.csv')
neutral_vocab =  pd.read_csv('neutralWords.csv')
 
positive_features = [(word_feats(pos), 'pos') for pos in positive_vocab]
negative_features = [(word_feats(neg), 'neg') for neg in negative_vocab]
neutral_features = [(word_feats(neu), 'neu') for neu in neutral_vocab]
 
train_set = negative_features + positive_features + neutral_features
classifier = NaiveBayesClassifier.train(train_set) 
 
# Predict
neg = 0
pos = 0
words = text.split()
for word in words:
    classResult = classifier.classify(word_feats(word))
    if classResult == 'neg':
        neg = neg + 1
    if classResult == 'pos':
        pos = pos + 1
print('Positive: ' + str(float(pos)/len(words)))
print('Negative: ' + str(float(neg)/len(words)))


Positive: 0.15613910574875797
Negative: 0.497101963567542


### 3a)

In [15]:
#!pip install chatterbot
#!pip install chatterbot_corpus

from chatterbot import ChatBot

In [16]:
# Create object of ChatBot class
bot = ChatBot('ChatBuddy')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\johnnyc\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\johnnyc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\johnnyc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
# Create object of ChatBot class with Storage Adapter
bot = ChatBot(
    'ChatBuddy',
    storage_adapter='chatterbot.storage.SQLStorageAdapter',
    database_uri='sqlite:///database.sqlite3'
)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\johnnyc\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\johnnyc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\johnnyc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
# Create object of ChatBot class with Logic Adapter
bot = ChatBot(
    'Buddy',  
    logic_adapters=[
        'chatterbot.logic.BestMatch'
        ],
)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\johnnyc\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\johnnyc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\johnnyc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
from chatterbot.trainers import ListTrainer

trainer = ListTrainer(bot)

trainer.train([
'Hi',
'Hello',
'Have a Good Day!'
])

List Trainer: [####################] 100%


In [20]:
# Get a response to the input text 'I would like to book a flight.'
response = bot.get_response('Hi, what are you')

print("Bot Response:", response)

No value for search_text was available on the provided input


Bot Response: I am good.


### 2b)

In [21]:
mathBot = ChatBot(
    'MathBot',
    logic_adapters=[
        'chatterbot.logic.MathematicalEvaluation'        
    ]
)

# Print an example of getting one math based response
response = mathBot.get_response('What is 4 * 9?')
print(response)

4 * 9 = 36


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\johnnyc\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\johnnyc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\johnnyc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
